In [18]:
import requests
from bs4 import BeautifulSoup

In [19]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://en.wikipedia.org/",
}

In [20]:
def fetch_html(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text

In [21]:

def scrape_team_logo(team_name: str, size_px: int = 256) -> str | None:
    url = "https://en.wikipedia.org/wiki/" + team_name.replace(" ", "_")
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    infobox = soup.find("table", class_="infobox")
    if not infobox:
        return None

    # ✅ target the dedicated logo cell
    logo_td = infobox.find("td", class_="infobox-image")
    if not logo_td:
        return None

    img = logo_td.find("img")
    if not img or not img.get("src"):
        return None

    src = img["src"]
    if src.startswith("//"):
        src = "https:" + src

    # upscale thumbnail when it's the standard /120px-... URL
    src = src.replace("/120px-", f"/{size_px}px-")
    return src